 # import libraries

In [4]:
import math

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import copy

from sklearn.metrics import (
    silhouette_score,
    pairwise_distances,
    mutual_info_score,
)

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from tslearn.metrics import cdist_dtw

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform

# functions

In [5]:

def z_normalize(T):
    return (T - np.mean(T))/np.std(T)

def replace_isolated_missed_value(T, isolated_missing_index):
    T[isolated_missing_index] = 0.5 * (
        T[isolated_missing_index - 1] + T[isolated_missing_index+1]
    )
    return T

In [6]:
def k_med_multi_centroid_Grouped_HighLevel_Featue_DTWenhanced(dict_of_data, k=30, n_itr=5, rs=0, 
                                                    dict_of_data_actual=None, Grouped_Features=None, keys2remove=[], w=1,
                                                    DTWmat_GroupedFeature_input=[], OneDay_flag=False, weighted=False, 
                                                    only_last_itr=True,
                                                    weighted_last_iteration = False):
    for key in keys2remove:
        del dict_of_data[key]
        del dict_of_data_actual[key]


    dict_of_data_keys = list(dict_of_data.keys())
    nkey = len(dict_of_data_keys)

    dict_of_data_keys_idx = {}
    for idx_mykey, my_key in enumerate(dict_of_data_keys):
        dict_of_data_keys_idx[my_key] = idx_mykey

  
    N, H = np.shape(dict_of_data[dict_of_data_keys[0]])
  
    # getting distance matices of different groups
    if len(DTWmat_GroupedFeature_input)==0:
        DTWdistmat_of_Grouped_Features = []
        for j_item, item in enumerate(Grouped_Features):
            third_dim = len(item)
            data = np.zeros((N,H,third_dim))
            dmat = 0
            for idx_data_key, data_key in enumerate(item):
                dmat += pairwise_distances(dict_of_data[data_key])
                data[:,:,idx_data_key] = dict_of_data[data_key]

            distmat_temp = cdist_dtw(data, global_constraint="sakoe_chiba", sakoe_chiba_radius=w) #dmat
            DTWdistmat_of_Grouped_Features.append(distmat_temp)
    else:
        DTWdistmat_of_Grouped_Features = DTWmat_GroupedFeature_input[:]


    # To investigate the impact of weighted approach, default: False
    if weighted:
        normalizer = [len(x) for x in Grouped_Features]
    else:
        normalizer = np.ones(len(DTWdistmat_of_Grouped_Features))

    if weighted_last_iteration:
        normalizer_inner = [len(x) for x in Grouped_Features]
    else:
        normalizer_inner = np.ones(len(DTWdistmat_of_Grouped_Features))


    print(f'normalizer: {normalizer}')

    ts_all = np.zeros((N,0))
    for ikey,key in enumerate(dict_of_data_keys):
        ts_all = np.concatenate((ts_all,dict_of_data[key]),axis=1)

    distmat_all = sum(np.square(x) for x in DTWdistmat_of_Grouped_Features)

    distmat_dict = {}
    for key in dict_of_data_keys:
        distmat_dict[key] = pairwise_distances(dict_of_data[key])

    km = KMedoids(n_clusters=k, init="k-medoids++")
    centroid_init = km._kpp_init(distmat_all, n_clusters=k, random_state_=check_random_state(rs))
    labels = np.argmin(distmat_all[centroid_init], axis=0)

    cmm_old = np.zeros((N,N), dtype='int')
    np.fill_diagonal(cmm_old, 1)

    labels_diff_vec = np.zeros(n_itr)
    inertia_tot_vec = np.zeros(n_itr)
    inertia_tot_per_item_vec = np.zeros((len(Grouped_Features), n_itr))


    intertia_tot_old = 0
    for itr in range(n_itr):
        center_id = np.empty((k,nkey), dtype='int')
        center_val = np.empty(shape=(k,H))
    
        inertia_tot = 0
        inertia_tot_per_item = np.zeros(len(Grouped_Features))
    
        for ik in range(k):
            days = np.ix_(labels==ik)[0]
            if len(days)==1:
                center_id[ik,:] = days[0]
                inertia_tot += 0

            else:
                for j_item, item in enumerate(Grouped_Features):
                    distmat = DTWdistmat_of_Grouped_Features[j_item]
                    distmat_sliced = distmat[np.ix_(days,days)]

                    # considering w=0 is just for the sake of consistency. (when w=0, it is basically ED!)
                    if w==0:
                        distmat_sliced_sq = np.square(distmat_sliced) #distmat_sliced 
                    else:
                        distmat_sliced_sq = np.square(distmat_sliced) #distmat_sliced np.square(distmat_sliced)


                    dist2others = np.sum(distmat_sliced_sq,axis=1)
                    sel_day =  days[np.argmin(dist2others)]
                    
                    # construct/update hybrid centroids
                    for key in item:
                        center_id[ik,dict_of_data_keys_idx[key]] = sel_day



                    inertia_tot_per_item[j_item] += np.min(np.sum(distmat_sliced_sq,axis=1))

                    inertia_tot += np.min(np.sum(distmat_sliced_sq,axis=1))
        

        inertia_tot_diff = inertia_tot - intertia_tot_old
        if abs(inertia_tot_diff)<0.001: # break update if the improvement is less than 0.001
            break

        intertia_tot_old = inertia_tot
        inertia_tot_per_item_vec[:,itr] = inertia_tot_per_item


        center_val = np.zeros((k,0))
        for ikey,key in enumerate(dict_of_data_keys):
            centers_id_of_key = center_id[:,ikey]    
            center_val = np.concatenate((center_val, dict_of_data[key][centers_id_of_key]), axis=1)

        OneDay_center_id = np.empty(k, dtype='int')
        if centroid_condition(n_itr, itr, only_last_itr, OneDay_flag):
            for ik in range(k):
                days = np.ix_(labels==ik)[0]
                if len(days)==1:
                    OneDay_center_id[ik] = days[0]
                else:
                    dvec = np.zeros(len(days))
                    for idx_d, d in enumerate(days):
                        dvec[idx_d] = dist_to_center_w(
                            dict_of_data,
                            dict_of_data_keys,
                            Grouped_Features,
                            DTWdistmat_of_Grouped_Features, 
                            center_id[ik,:],
                            d,
                            dict_of_data_keys_idx,
                            normalizer_inner
                        )


                OneDay_center_id[ik] = days[np.argmin(dvec)]

            D = pairwise_distances(ts_all , ts_all[OneDay_center_id]) 

        else:
            D = np.zeros((N,k))
            for ik in range(k):
                for j_item, item in enumerate(Grouped_Features):
                    distmat = DTWdistmat_of_Grouped_Features[j_item]

                    if w==0:
                        distmat_sq = np.square(distmat) #distmat
                    else:
                        distmat_sq = np.square(distmat) #distmat
                    sel_day = int(center_id[ik,dict_of_data_keys_idx[item[0]]])
                    D[:,ik] += distmat_sq[:,sel_day]/normalizer[j_item]

        labels = np.argmin(D,axis=1)
        cmm_new = cmm_creator(labels)
        labels_diff_vec[itr] = np.sum(np.abs(cmm_new - cmm_old))

        cmm_old = cmm_new
        inertia_tot_vec[itr] = inertia_tot
    
    dict4dc = dict_of_data_actual
    
    # calc errors for the sake of on-line investigation 
    err_dc_dict={}
    for ikey, key in enumerate(dict_of_data_keys):
        if OneDay_flag:
            my_center_id = OneDay_center_id[:]
        else:
            my_center_id = center_id[:,ikey]

        data_replica = reconstruct_data(dict4dc[key],labels, my_center_id)
        err=err_of_dc_grouply(dict4dc[key],data_replica, labels, my_center_id, day_wise=False, normalized=True, use_normalizd_form=False)

        err_dc_dict[key] = err
    
    # force to avoid hybrid (default: False).
    # this is for the sake of investigation
    if OneDay_flag:
        nr,nc = center_id.shape
        for c in range(nc):
            center_id[:,c] = OneDay_center_id
    
    # To handle cases when it stuck into a local optima loop
    if np.abs(inertia_tot_diff) > 0.001:
        inertia_tot_vec[-1] = 40000


    return ts_all, labels, center_id, dict_of_data_keys, inertia_tot_vec[-1], err_dc_dict, DTWdistmat_of_Grouped_Features, OneDay_center_id

In [7]:
# hierarchical clustering with min-max linkage 
# this is the second stage of KmHC
# when w=0, it is basically ED!
def HC_MinMax_klst(data, k_arr, n_init=None, w=0, flag_radius_avg=False,flag_distmat=False,given_distmat=None):
    
    shape_info = list(np.shape(data))
    N = shape_info[0]
    H = shape_info[1]
    D = 0 if len(shape_info)<3 else shape_info[-1]
    
    if not flag_distmat:
        if w>0:
            distmatall = cdist_dtw(data,global_constraint='sakoe_chiba',sakoe_chiba_radius=w)
        else:
            distmatall = pairwise_distances(data)
    else:
        distmatall = given_distmat
        
    distmat_group = copy.deepcopy(distmatall)
    
    cmm = np.eye(N)
    groups = find_groups_from_cmm(cmm)
    ng = len(groups)

    nk = len(k_arr)    
    labels_mat = np.zeros((nk,N))
      
    cnt=0
    k_min = np.min(k_arr)
    while ng>k_min:
        groups_keys = list(groups.keys())
         
        np.fill_diagonal(distmat_group,float('inf'))
        closest_dist = np.min(distmat_group,axis=1)
        closest_dist_ind = np.argmin(distmat_group,axis=1)      
        ind0 = np.argmin(closest_dist)
        ind1 = closest_dist_ind[ind0]
            
        key0 = groups_keys[ind0]
        key1 = groups_keys[ind1]
        
        days0 = np.array(groups[key0])
        days1 = np.array(groups[key1])
        
        days_joined = np.reshape(np.row_stack((days0.reshape(-1,1),days1.reshape(-1,1))),newshape=(-1,))
        days_joined = [x for x in days_joined]
        
        del groups[key0]
        del groups[key1]
        
        new_key = 1000 + cnt
        groups[new_key] = days_joined
        groups_keys = list(groups.keys())
        
        distmat_group = np.delete(distmat_group,[ind0,ind1],axis=0)
        distmat_group = np.delete(distmat_group,[ind0,ind1],axis=1)
        
        distmat_group_new = np.zeros((ng-1,ng-1))
        distmat_group_new[:-1,:-1] = distmat_group
        
        for i in range(ng-2):
            days0 = np.array(groups[groups_keys[i]])
            days1 = np.array(groups[new_key])
            days_joined = np.reshape(np.row_stack((days0.reshape(-1,1),days1.reshape(-1,1))),newshape=(-1,))
            days_joined = [x for x in days_joined]
            
            if not flag_radius_avg:
                val = np.min(np.max(distmatall[np.ix_(days_joined,days_joined)],axis=1))
                         
            distmat_group_new[-1,i] = val
            distmat_group_new[i,-1] = val
        
        distmat_group_new[-1,-1] = float('inf')
        distmat_group = distmat_group_new
        
        ng = len(distmat_group)
        
        
        
        if ng in k_arr:
            index_of_ng = np.ix_(k_arr==ng)[0][0]
            for i,key in enumerate(groups):
                for d in groups[key]:
                    labels_mat[index_of_ng,d] = i
    
        
        cnt += 1
        
    return labels_mat


In [8]:
#some func to measure edc:

# for on-line investigation and analysis (can be skipped)
def err_of_dc_grouply(data,data_replica, label, center_id, day_wise=False, normalized=True, use_normalizd_form=False):
    
    kl=len(set(label))
    err_vec=np.zeros(kl)
    err_vec_daily=np.zeros(np.shape(data)[0])
    t=8760
    
    if use_normalizd_form==False:
        size_vec=np.zeros(kl)
      
        if day_wise is not None:
            for ik in range(kl):
                days=np.ix_(label==ik)[0]
                if len(days)==1:
                    err_vec[ik]=0
                    err_vec_daily[days[0]]=0
                else:
                    if day_wise==False:
                        arr1 = np.sort(np.reshape(data[np.ix_(days,)],newshape=(-1,)))[::-1]
                        arr2 = np.sort(np.reshape(data_replica[np.ix_(days,)],newshape=(-1,)))[::-1]
                        err_vec[ik] = err_func(arr1,arr2)

                    elif day_wise==True:
                        for d in days:
                            arr1 = np.sort(np.reshape(data[d,],newshape=(-1,)))[::-1]
                            arr2 = np.sort(np.reshape(data_replica[d,],newshape=(-1,)))[::-1]

                            err_vec_daily[d] =  err_func(arr1,arr2)

            if day_wise==False:
                err=err_vec
            elif day_wise==True:
                err=err_vec_daily

            data_vecform = data.reshape(-1,)
            if normalized:
                denom = np.abs(np.max(data_vecform)-np.min(data_vecform)) #np.mean(np.abs(data.reshape(-1,))): main results for paper
            else:
                denom = 1

            return ((np.sum(np.square(err))/t)**0.5)/denom

        else:
            arr1 = np.sort(np.reshape(data,newshape=(-1,)))[::-1]
            arr2 = np.sort(np.reshape(data_replica,newshape=(-1,)))[::-1]
            err =  err_func(arr1,arr2)
            return (((np.square(err))/t)**0.5)/np.mean(np.abs(data.reshape(-1,)))

    elif use_normalizd_form==True:
        data_replica = reconstruct_data(data,label, center_id) #data in the input argument is supposed to be yr_normed 


        for ik in range(kl):
            days=np.ix_(label==ik)[0]
            if len(days)==1:
                err_vec[ik]=0
                err_vec_daily[days[0]]=0

            else:
                if day_wise==False:
                    arr1 = np.sort(np.reshape(data[np.ix_(days,)],newshape=(-1,)))[::-1]
                    arr2 = np.sort(np.reshape(data_replica[np.ix_(days,)],newshape=(-1,)))[::-1]

                    err_vec[ik] = err_func(arr1,arr2)

                elif day_wise==True:        
                    for d in days:
                        arr1 = np.sort(np.reshape(data[d,],newshape=(-1,)))[::-1]
                        arr2 = np.sort(np.reshape(data_replica[d,],newshape=(-1,)))[::-1]
                        err_vec_daily[d] =  err_func(arr1,arr2)


        if day_wise==False:
            err=err_vec
        elif day_wise==True:
            err=err_vec_daily

        return (np.sum(np.square(err))/t)**0.5


# reconstruct data based on aggregated data
def reconstruct_data(data,label_lst, centers_id, center_val=None):
    
    data_reconstruct=np.zeros_like(data)
    for ind in range(len(label_lst)):
        if center_val is None:
            data_reconstruct[ind,:] = data[centers_id[label_lst[ind]],:]
        else:
            data_reconstruct[ind,:] = center_val[label_lst[ind],:]
    
    return data_reconstruct


# a dc-to-dc error
def err_func(arr_sort_1,arr_sort_2):
    return np.linalg.norm(arr_sort_1-arr_sort_2)



# find the size of clusters using the labels
def clust_size(labels_arr):
    klab = len(set(labels_arr))
    clust_size_arr = np.empty(klab, dtype='int')
    for ik in range(klab):
        objs = np.ix_(labels_arr==ik)[0]
        clust_size_arr[ik] = len(objs)
    
    return clust_size_arr

# Data

In [9]:
#data directory
data_dir = ".//2019//"

In [ ]:
# read demand data
df_Demand = pd.read_csv(data_dir + 'AB_demand_2019.csv')

# convert to numpy array
ts_Demand = df_Demand['AB - Alberta Internal Load Hr Avg MW'].to_numpy()

###############################

# handling missing values can be automated. It is hard-coded here.

# fill in missing value(s) by considering nearby values
ts_Demand[1634] = (ts_Demand[1633]+ts_Demand[1635])/2 

# actual data in 2D
ts_Demand_2d = np.reshape(ts_Demand, newshape=(365,24) )

# z-norm of actual data and then tranform it to 2D
ts_Demand_norm = z_normalize(ts_Demand)
ts_Demand_norm_2d = np.reshape(ts_Demand_norm, newshape=(365,24) )

In [14]:
# read wind power data
df_WFs = pd.read_csv(data_dir + 'AB_windpower_2019.csv') 
# df_WFs.describe()

In [15]:
# read solar data
df_solar = pd.read_csv(data_dir + 'AB_solar_2019.csv') # E:\second_paper\DATA_To_Study\2019
# df_solar.describe()

In [17]:
df_load = pd.read_csv(data_dir + 'AB_demand_2019.csv')
ts_load = df_load['AB - Alberta Internal Load Hr Avg MW'].to_numpy()
ts_solar = df_solar['MW'].to_numpy()


In [18]:
ts_BSR1_Center = df_WFs['AB - Blackspring Ridge Hr Avg MW'].to_numpy()
ts_HAL1_CE = df_WFs['AB - Halkirk Wind Power Facility Hr Avg MW'].to_numpy()
ts_BTR1_SW = df_WFs['AB - Blue Trail Wind Hr Avg MW'].to_numpy()
ts_McBrideLake_SW = df_WFs['AB - McBride Lake Hr Avg MW'].to_numpy()

#extra wf data:
ts_wf_ext1 = df_WFs['AB - Ardenville Wind Hr Avg MW'].to_numpy()
ts_wf_ext2 = df_WFs['AB - Castle Rock Wind Farm Hr Avg MW'].to_numpy()
ts_wf_ext3 = df_WFs['AB - Kettles Hill Wind Hr Avg MW'].to_numpy()
ts_wf_ext4 = df_WFs['AB - Soderglen Wind Hr Avg MW'].to_numpy()
ts_wf_ext5 = df_WFs['AB - Suncor Wintering Hills Hr Avg MW'].to_numpy()
ts_wf_ext6 = df_WFs['AB - Enmax Taber Hr Avg MW'].to_numpy()

# in the paper, we considered 

In [20]:
# handle missing value 
# note: 1634 is the index with missing value 
# it is hard coded. To automate this, use pd.isnull on the csv 

ts_BSR1_Center = replace_isolated_missed_value(ts_BSR1_Center, 1634)
ts_HAL1_CE = replace_isolated_missed_value(ts_HAL1_CE, 1634)
ts_BTR1_SW = replace_isolated_missed_value(ts_BTR1_SW, 1634)
ts_McBrideLake_SW = replace_isolated_missed_value(ts_McBrideLake_SW, 1634)

ts_load = replace_isolated_missed_value(ts_load, 1634)


ts_wf_ext1 = replace_isolated_missed_value(ts_wf_ext1, 1634)
ts_wf_ext2 = replace_isolated_missed_value(ts_wf_ext2, 1634)
ts_wf_ext3= replace_isolated_missed_value(ts_wf_ext3, 1634)
ts_wf_ext4 = replace_isolated_missed_value(ts_wf_ext4, 1634)
ts_wf_ext5 = replace_isolated_missed_value(ts_wf_ext5, 1634)
ts_wf_ext6 = replace_isolated_missed_value(ts_wf_ext6, 1634)

# note: ts_solar has no missing value.

In [21]:
# load
ts_load = np.copy(ts_load)

# solar
ts_solar = np.copy(ts_solar)

# wind
ts_Center = np.copy(ts_BSR1_Center)
ts_CE = np.copy(ts_HAL1_CE)

ts_SW_1 = np.copy(ts_BTR1_SW)
ts_SW_2 = np.copy(ts_McBrideLake_SW)

ts_wf_ext1 = np.copy(ts_wf_ext1)
ts_wf_ext2 = np.copy(ts_wf_ext2)
ts_wf_ext3 = np.copy(ts_wf_ext3)
ts_wf_ext4 = np.copy(ts_wf_ext4)
ts_wf_ext5 = np.copy(ts_wf_ext5)
ts_wf_ext6 = np.copy(ts_wf_ext6)

# create dict of data
dict_data1d_actual = {'load': ts_load, 
                      'solar': ts_solar,
                      
                      'Center':ts_Center, 
                      'CE': ts_CE, 
                      'SW_1': ts_SW_1, 
                      'SW_2': ts_SW_2, 
                      
                      'wf_ext1':ts_wf_ext1,
                      'wf_ext2':ts_wf_ext2,
                      'wf_ext3':ts_wf_ext3,
                      'wf_ext4':ts_wf_ext4,
                      'wf_ext5':ts_wf_ext5,
                      'wf_ext6':ts_wf_ext6
                      }

In [22]:
# create 2D arrays, each row corresponds to one day of year

ts_load_minmax = (np.reshape(ts_load, newshape=(365,24))-np.min(ts_load))/(np.max(ts_load)-np.min(ts_load))
ts_load_MAXscaled = np.reshape(ts_load, newshape=(365,24))/(np.max(ts_load))

ts_solar_minmax = np.reshape(ts_solar, newshape=(365,24))/np.max(ts_solar)

ts_Center_minmax = (np.reshape(ts_Center, newshape=(365,24)))/np.max(ts_Center)
ts_CE_minmax = (np.reshape(ts_CE,  newshape=(365,24)))/np.max(ts_CE)

ts_SW_1_minmax = np.reshape(ts_SW_1, newshape=(365,24))/np.max(ts_SW_1)
ts_SW_2_minmax = np.reshape(ts_SW_2, newshape=(365,24))/np.max(ts_SW_2)

ts_wf_ext1_minmax = np.reshape(ts_wf_ext1, newshape=(365,24))/np.max(ts_wf_ext1)
ts_wf_ext2_minmax = np.reshape(ts_wf_ext2, newshape=(365,24))/np.max(ts_wf_ext2)
ts_wf_ext3_minmax = np.reshape(ts_wf_ext3, newshape=(365,24))/np.max(ts_wf_ext3)
ts_wf_ext4_minmax = np.reshape(ts_wf_ext4, newshape=(365,24))/np.max(ts_wf_ext4)
ts_wf_ext5_minmax = np.reshape(ts_wf_ext5, newshape=(365,24))/np.max(ts_wf_ext5)
ts_wf_ext6_minmax = np.reshape(ts_wf_ext6, newshape=(365,24))/np.max(ts_wf_ext6)




In [23]:
ts_load_norm = np.reshape(z_normalize(ts_load), newshape=(365,24))
ts_solar_norm = np.reshape(z_normalize(ts_solar), newshape=(365,24))


# the wind farms ext1 and ext4 will be excluded as they have faulty data.
ts_Center_norm = np.reshape(z_normalize(ts_Center), newshape=(365,24))
ts_CE_norm = np.reshape(z_normalize(ts_CE),  newshape=(365,24))

ts_SW_1_norm = np.reshape(z_normalize(ts_SW_1), newshape=(365,24))
ts_SW_2_norm = np.reshape(z_normalize(ts_SW_2), newshape=(365,24))


ts_wf_ext1_norm = np.reshape(z_normalize(ts_wf_ext1), newshape=(365,24))
ts_wf_ext2_norm = np.reshape(z_normalize(ts_wf_ext2), newshape=(365,24))
ts_wf_ext3_norm = np.reshape(z_normalize(ts_wf_ext3), newshape=(365,24))
ts_wf_ext4_norm = np.reshape(z_normalize(ts_wf_ext4), newshape=(365,24))
ts_wf_ext5_norm = np.reshape(z_normalize(ts_wf_ext5), newshape=(365,24))
ts_wf_ext6_norm = np.reshape(z_normalize(ts_wf_ext6), newshape=(365,24))

In [24]:
dict_data_norm = {'load':ts_load_norm, 
                  'solar':ts_solar_norm,

                  'Center':ts_Center_norm, 
                  'CE': ts_CE_norm, 
                  
                  #'SW': ts_SW_norm,
                  'SW_1': ts_SW_1_norm,
                  'SW_2': ts_SW_2_norm, 
                  
                  'wf_ext1':ts_wf_ext1_norm,
                  'wf_ext2':ts_wf_ext2_norm,
                  'wf_ext3':ts_wf_ext3_norm,
                  'wf_ext4':ts_wf_ext4_norm,
                  'wf_ext5':ts_wf_ext5_norm,
                  'wf_ext6':ts_wf_ext6_norm
                  
                  }







dict_data_minmax = {'load':ts_load_minmax, 
                    'solar': ts_solar_minmax,
                    
                    'Center':ts_Center_minmax, 
                    'CE': ts_CE_minmax, 
                    
                    #'SW': ts_SW_minmax,
                    'SW_1': ts_SW_1_minmax,
                    'SW_2': ts_SW_2_minmax,

                    
                    
                    
                    
                    'wf_ext1':ts_wf_ext1_minmax,  
                    'wf_ext2':ts_wf_ext2_minmax,
                    'wf_ext3':ts_wf_ext3_minmax,
                    'wf_ext4':ts_wf_ext4_minmax, 
                    'wf_ext5':ts_wf_ext5_minmax,
                    'wf_ext6':ts_wf_ext6_minmax
                      
                    }

In [25]:
dict2use = dict_data_norm  #dict_data_disc_norm #dict_data_norm
dict2use_keys = list(dict2use.keys())

distmat_dict={}
distmat_lst=[]

cntr=0
for key in dict2use_keys:
    cntr += 1
    distmat_dict[key] = pairwise_distances(dict2use[key])


In [ ]:
# remove two wfs due to their faulty data
keys2ues = dict2use_keys[:]
keys2ues.remove('wf_ext1')
keys2ues.remove('wf_ext4')


# Grouping resources

In [114]:
klst = [20]

In [ ]:
rs = 0
k_labels = {}

for jk, k in enumerate(klst):
    k_labels[k] = {}
    # print(f">>> k={k}")
    for  i_key, key in enumerate(keys2ues):
        # print(f"key={key}")
        mydistmat = distmat_dict[key]
        km = KMeans(n_clusters=k, init='k-means++', max_iter=100,  n_init=50, random_state = rs).fit(dict2use[key])
    
        k_labels[k][key] = km.labels_

In [38]:
n_datasets = len(keys2ues)

dict_distmat_between_datasets = {}

for jk, k in enumerate(klst):
    dict_distmat_between_datasets[k] = np.zeros((n_datasets,n_datasets))
    for i in range(n_datasets-1):
        ikey = keys2ues[i]
        for j in range(i+1,n_datasets):
            jkey = keys2ues[j]
            I = mutual_info_score(k_labels[k][ikey], k_labels[k][jkey])
            d = np.exp(-I)

            dict_distmat_between_datasets[k][i,j] = d
            dict_distmat_between_datasets[k][j,i] = d

In [46]:
linkage2use=  'average'


v = 0 # because  we have just one k here in this case
k = klst[v]

distmat = dict_distmat_between_datasets[k]
n = np.shape(distmat)[0]

dists = squareform(distmat)
linkage_matrix = linkage(dists, linkage2use)

n_feature = np.shape(distmat)[0] # n_feature : number of resources

plot_dendogram = False
if plot_dendogram:
    plt.figure(figsize=(10,5))
    dendrogram(linkage_matrix, labels=[str(x) for x in range(0,n_feature)], orientation='top')
    plt.title("dendrogram")
    plt.show()

In [47]:
max_compactness_list = []
maxdist_vec = [] 
within_sum_lst = []
silscore_vec = []

for id_f in range(1,n_feature+1):
    within_sum = 0
    hc = AgglomerativeClustering(id_f, affinity='precomputed', linkage=linkage2use).fit(distmat)
    f=hc.labels_
    
    if ((id_f>1) and (id_f<n_feature)):
        silscore = silhouette_score(distmat,f, metric='precomputed')
    else:
        silscore = 0
    silscore_vec.append(silscore)
    f_unique = len(set(f))
    
    max_dist = 0
    for f_index in range(f_unique):
        idx = np.ix_(f==f_index)[0]
        distmat_slice = distmat[np.ix_(idx,idx)]
        max_dist = max(max_dist, np.max(distmat_slice))
    
    maxdist_vec.append(max_dist)



fs = 13 # fontsize
s = silscore_vec[1:-1]

plot_silhoutte = False
if plot_silhoutte:
    plt.plot(s)
    plt.xticks(ticks=np.arange(8), labels=np.arange(2,10),fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.xlabel('G (Number of Groups)',fontsize=fs)
    plt.ylabel('Silhoutte Score',fontsize=fs)
    plt.axvline(x=2, ymax = 0.95, linestyle='--', color='r')
    plt.scatter(x=2,y=s[2], color='r')
    plt.tight_layout()
    plt.show()

In [48]:
# labels we used in the paper
keys4plot_paper = ['D', 'SF', 'WF(3)', 'WF(1)', 'WF(5)','WF(6)','WF(7)','WF(8)', 'WF(2)', 'WF(4)']

In [49]:
nclust=4 # number of groups created by grouping data resources (see graph above)

distmat = dict_distmat_between_datasets[k]
dists = squareform(distmat)

linkage_matrix = linkage(dists, linkage2use )

f = fcluster(linkage_matrix,nclust,criterion='maxclust')

lst = []
for id in list(set(f)):
    lst.append(np.where(f==id)[0].tolist())

# print('grouped data as: ', lst)
# print('dict2use_keys: \n', dict2use_keys)

# print('ID of each data set: \n', [(idx,idx_x) for idx, idx_x in enumerate(keys2ues)])

In [54]:
grouped_highlevel_features=[]
for item in lst:
    grouped_data = []
    for g in item:
        grouped_data.append(keys2ues[g])
  
    grouped_highlevel_features.append(grouped_data)
    
# print('grouped_highlevel_features: \n', grouped_highlevel_features)

In [56]:
grouped_highlevel_features

[['load', 'solar'],
 ['CE', 'wf_ext5'],
 ['SW_1', 'SW_2', 'wf_ext2', 'wf_ext3'],
 ['Center', 'wf_ext6']]

# proposal M-RD

In [57]:
OneDay_flag = False
only_last_itr = False
weighted_last_iteration = False

weighted = False

n_randomstates = 1000

k=20; w=2; 
# setting w=0 means we are doing ED!
# setting w=1 means we are just doing one step head mapping, not too much advantage!
# setting w=2, about 10% of length of observation (24) is reasonable [see Eamon Keogh Tutorials]

adjustment = False

In [59]:
keys2remove = ['wf_ext1', 'wf_ext4']

In [69]:

ts_load_actual_vals =  np.reshape(ts_load, newshape=(365,24))

ts_solar_actual_vals = np.reshape(ts_solar, newshape=(365,24))


ts_Center_actual_vals = np.reshape(ts_Center, newshape=(365,24))
ts_CE_actual_vals = np.reshape(ts_CE, newshape=(365,24))

ts_SW_1_actual_vals = np.reshape(ts_SW_1, newshape=(365,24))
ts_SW_2_actual_vals = np.reshape(ts_SW_2, newshape=(365,24))


ts_wf_ext1_actual_vals = np.reshape(ts_wf_ext1, newshape=(365,24))
ts_wf_ext2_actual_vals = np.reshape(ts_wf_ext2, newshape=(365,24))
ts_wf_ext3_actual_vals = np.reshape(ts_wf_ext3, newshape=(365,24))
ts_wf_ext4_actual_vals = np.reshape(ts_wf_ext4, newshape=(365,24))
ts_wf_ext5_actual_vals = np.reshape(ts_wf_ext5, newshape=(365,24))
ts_wf_ext6_actual_vals = np.reshape(ts_wf_ext6, newshape=(365,24))





dict_data2d_actual = {'load': ts_load_actual_vals,
                      'solar': ts_solar_actual_vals,

                      'Center':ts_Center_actual_vals, 
                      'CE': ts_CE_actual_vals, 
                      
                      'SW_1': ts_SW_1_actual_vals,
                      'SW_2': ts_SW_2_actual_vals,

                      'wf_ext1':ts_wf_ext1_actual_vals,
                      'wf_ext2':ts_wf_ext2_actual_vals,
                      'wf_ext3':ts_wf_ext3_actual_vals,
                      'wf_ext4':ts_wf_ext4_actual_vals,
                      'wf_ext5':ts_wf_ext5_actual_vals,
                      'wf_ext6':ts_wf_ext6_actual_vals
                      }



# print('min per data set')
# for key in dict_data2d_actual.keys():
#   print(f'at key={key}, minimum is: {np.min(dict_data2d_actual[key])}')

In [ ]:
print(f'w={w}, and k={k}, OneDay_flag={OneDay_flag}')
print('----------------')

rs_lst = np.arange(n_randomstates)
inertia_lst = np.zeros(len(rs_lst))

DTWmat_GroupedFeature = []
for rs in rs_lst:
    (
        _, _, _,_, inertia_tot, _, DTWmat_GroupedFeature, OneDay_Centroid
    ) = k_means_multi_centroid_Grouped_HighLevel_Featue_DTWenhanced(
        copy.deepcopy(dict_data_norm),  
        k=k, 
        n_itr=50, 
        rs=rs, 
        dict_of_data_actual=copy.deepcopy(dict_data2d_actual), 
        Grouped_Features=grouped_highlevel_features, 
        keys2remove=keys2remove, 
        w=w, 
        DTWmat_GroupedFeature_input = DTWmat_GroupedFeature, 
        OneDay_flag=OneDay_flag, 
        weighted=weighted,
        only_last_itr=only_last_itr,
        weighted_last_iteration = weighted_last_iteration
    )
    
    inertia_lst[rs] = inertia_tot

In [ ]:
rs = np.argmin(np.abs(inertia_lst))
# print('random state with minimum inertia: ', rs)

In [62]:
dict_of_data = copy.deepcopy(dict_data_norm) 
keys2remove = keys2remove

for key in keys2remove:
    del dict_of_data[key]

dict_of_data_keys = list(dict_of_data.keys())

dict_pseudokey2actual = {'load':'Load',
                         'solar':'Solar',
                          
                          'Center':'Blackspring', 
                         'CE': 'Halkirk',
                         'SW_1':'Blue_Trail',
                         'SW_2':'McBride',
                         
                         'wf_ext2': 'Castle_Rock_Wind_Farm',
                         'wf_ext3':'Kettles_Hill',
                         'wf_ext5':'Wintering_Hills',
                         'wf_ext6':'Enmax'}

dict_of_data_keys_ACTUAL = [dict_pseudokey2actual[x] for x in dict_of_data_keys]

In [ ]:
# print(f'w={w}, and k={k}, and rs={rs}')
# print('grouped_highlevel_features: \n', grouped_highlevel_features)

(
    ts_all, 
    labels_proposal, 
    center_id_proposal, 
    dict_of_data_keys, 
    inertia_tot, 
    err_dc_dict, 
    DTWmat_GroupedFeature, 
    OneDay_Centroid
) = k_med_multi_centroid_Grouped_HighLevel_Featue_DTWenhanced(
    copy.deepcopy(dict_data_norm), 
    k=k, 
    n_itr=25, 
    rs=rs, 
    dict_of_data_actual=copy.deepcopy(dict_data2d_actual), 
    Grouped_Features=grouped_highlevel_features, 
    keys2remove=keys2remove, 
    w=w,
    DTWmat_GroupedFeature_input = [],
    OneDay_flag=OneDay_flag, 
    weighted = weighted,
    only_last_itr=only_last_itr,
    weighted_last_iteration=weighted_last_iteration
)

In [ ]:
if OneDay_flag:
    center_id_proposal = np.empty_like(center_id_proposal)
    for f in range(center_id_proposal.shape[1]):
        center_id_proposal[:,f] = OneDay_Centroid

df_centers_id = pd.DataFrame(center_id_proposal, columns=dict_of_data_keys_ACTUAL, index=np.arange(k))
# save
df_centers_id.to_csv('center_id_proposal.csv')


df_label=pd.Series(labels_proposal, name='Labels')
# save
df_label.to_csv('labels_proposal.csv')

labels = copy.deepcopy(labels_proposal)
center_id = copy.deepcopy(center_id_proposal)

for ikey, key in enumerate(dict_of_data_keys):
    actual_name = dict_pseudokey2actual[key]
    # print(f'key={key} >>> actual_name: {actual_name}')
    N, H = np.shape(dict_data_norm[key])

    if key=='load':
        dat2use = ts_load_MAXscaled
    else:
        dat2use = dict_data_minmax[key]

    data_reconstructed = reconstruct_data(dat2use, labels, center_id[:,ikey], center_val=None)
    data_reconstructed_rep = data_reconstructed[center_id[:,ikey],:]
  
    if adjustment:
        data_reconstructed_rep = adjusting_rep (dat2use, data_reconstructed_rep, labels)
    
    # NOTE: create folder f"proposal_k{k}" first! 
    df = pd.DataFrame(data_reconstructed_rep, columns=np.arange(1,25), index=np.arange(1,k+1))
    df.to_csv(f'proposal_k{k}/{actual_name}.csv')
  
    weights_of_rep = clust_size(labels)
    df_weights = pd.Series(weights_of_rep, name='weights_per_cluster')
    
    # save
    df_weights.to_csv('weights_per_cluster.csv')

# K-Medoids

In [71]:
k=20
w=0 

#NOTE: w is the window constraint in DTW. Hence, w=0 basically means ED!!!

In [75]:
N, H = np.shape(dict_of_data[dict_of_data_keys[0]])
# N: number of days in the year

In [77]:
dict_of_data = copy.deepcopy(dict_data_norm)
keys2remove = keys2remove

for key in keys2remove:
    del dict_of_data[key]

dict_of_data_keys = list(dict_of_data.keys())
# print('dict_of_data_keys: \n', dict_of_data_keys)

keys2remove = keys2remove


dict_pseudokey2actual = {'load':'Load',
                         'solar':'Solar',
                          
                          'Center':'Blackspring', 
                         'CE': 'Halkirk',
                         'SW_1':'Blue_Trail',
                         'SW_2':'McBride',
                         
                         'wf_ext2': 'Castle_Rock_Wind_Farm',
                         'wf_ext3':'Kettles_Hill',
                         'wf_ext5':'Wintering_Hills',
                         'wf_ext6':'Enmax'}


dict_of_data_keys_ACTUAL = [dict_pseudokey2actual[x] for x in dict_of_data_keys]

# print('dict_pseudokey2actual: \n', dict_pseudokey2actual)

ts_all = np.zeros((N,0))
for ikey,key in enumerate(dict_of_data_keys):
    ts_all = np.concatenate((ts_all,dict_of_data[key]),axis=1)

In [78]:
n_randomstates=1000

In [ ]:
rs_lst = np.arange(n_randomstates)
inertia_lst = np.zeros(len(rs_lst))

distmat_ts_all = pairwise_distances(ts_all)

for rs in rs_lst:
    kmedoids = KMedoids(n_clusters=k, random_state=rs,  max_iter=50, method='pam', init='random').fit(ts_all) # init='k-medoids++'
    inertia_lst[rs] = kmedoids.inertia_
    silscore_lst[rs] = silhouette_score(distmat_ts_all, kmedoids.labels_, metric='precomputed')
    del kmedoids

rs=np.argmin(inertia_lst)
rs = rs_candid[np.argmax(silscore_lst_candid)]

kmedoids = KMedoids(n_clusters=k, random_state=rs,  max_iter=50, method='pam', init='random').fit(ts_all)
kmedoids_lab = kmedoids.labels_
kmedoids_centers_id = kmedoids.medoid_indices_

keys2remove = keys2remove


dict4dc = copy.deepcopy(dict_data2d_actual)
for key in keys2remove:
    del dict4dc[key]
dict4dc_key = list(dict4dc.keys())

kmedoids_centers_id_mat=np.empty(shape=(k,len(dict4dc_key)),dtype='int')
for i in range(len(dict4dc_key)):
    kmedoids_centers_id_mat[:,i]=kmedoids_centers_id


err_dc_dict={}
for ikey, key in enumerate(dict4dc_key):
    data_replica = reconstruct_data(dict4dc[key],kmedoids_lab, kmedoids_centers_id)
    err=err_of_dc_grouply(dict4dc[key],data_replica, kmedoids_lab, kmedoids_centers_id, day_wise=False, normalized=True, use_normalizd_form=False)
    err_dc_dict[key] = err


err_regular = copy.deepcopy(err_dc_dict)
df_err=pd.DataFrame([err_regular])
df_err.to_csv('err_dc.csv')

df_centers_id = pd.DataFrame(kmedoids_centers_id_mat, columns=dict_of_data_keys_ACTUAL, index=np.arange(k))
df_centers_id.to_csv('center_id_kmed.csv')

df_label=pd.Series(kmedoids_lab, name='Labels')
df_label.to_csv('labels_kmed.csv')


labels = copy.deepcopy(kmedoids_lab)
center_id = copy.deepcopy(kmedoids_centers_id_mat)

for ikey, key in enumerate(dict_of_data_keys):
    actual_name = dict_pseudokey2actual[key]
    
    
    if key=='load':
        dat2use = ts_load_MAXscaled
    else:
        dat2use = dict_data_minmax[key]



    data_reconstructed = reconstruct_data(dat2use, labels, center_id[:,ikey], center_val=None)
    data_reconstructed_rep = data_reconstructed[center_id[:,ikey],:]
  
    df = pd.DataFrame(data_reconstructed_rep, columns=np.arange(1,25), index=np.arange(1,k+1))
    
    # NOTE: create folder f"kmed_k{k}"
    df.to_csv(f'kmed_k{k}'+f'/{actual_name}.csv')
  
  
    weights_of_rep = clust_size(labels)
    df_weights = pd.Series(weights_of_rep, name='weights_per_cluster')
    df_weights.to_csv('weights_per_cluster.csv')

# kmean_close2avg

In [79]:
k=20 
w=0 
# NOTE: w=0 in DTW basically means ED! this is just to be on the safe side and use ED even if we try to use DTW here.

In [81]:
dict_of_data = copy.deepcopy(dict_data_norm)
keys2remove = keys2remove

for key in keys2remove:
    del dict_of_data[key]

dict_of_data_keys = list(dict_of_data.keys())
# print('dict_of_data_keys: \n', dict_of_data_keys)

dict_pseudokey2actual = {'load':'Load',
                         'solar':'Solar',
                          
                          'Center':'Blackspring', 
                         'CE': 'Halkirk',
                         'SW_1':'Blue_Trail',
                         'SW_2':'McBride',
                         
                         'wf_ext2': 'Castle_Rock_Wind_Farm',
                         'wf_ext3':'Kettles_Hill',
                         'wf_ext5':'Wintering_Hills',
                         'wf_ext6':'Enmax'}


dict_of_data_keys_ACTUAL = [dict_pseudokey2actual[x] for x in dict_of_data_keys]
# print('dict_pseudokey2actual: \n', dict_pseudokey2actual)

ts_all = np.zeros((N,0))
for ikey,key in enumerate(dict_of_data_keys):
    ts_all = np.concatenate((ts_all,dict_of_data[key]),axis=1)

In [83]:
n_randomstates=1000

In [ ]:
# print('shape of ts_all: \n', np.shape(ts_all))
# print('=====================================')

for rs in rs_lst:
    km = KMeans(n_clusters=k, random_state=rs).fit(ts_all)
    inertia_lst[rs] = km.inertia_

rs = np.argmin(inertia_lst)

km = KMeans(n_clusters=k, random_state=rs).fit(ts_all)
km_lab = km.labels_
km_center = np.squeeze(km.cluster_centers_, axis=2)

keys2remove = keys2remove
dict4dc = copy.deepcopy(dict_data2d_actual)
for key in keys2remove:
    del dict4dc[key]
dict4dc_key = list(dict4dc.keys())


km_center_id = np.empty(k, dtype='int')

for ik in range(k):
    days = np.ix_(km_lab==ik)[0]
    days = days.tolist()
    ts_slice = ts_all[days,:]

    D = pairwise_distances(np.reshape(km_center[ik,:], newshape=(1,-1)), ts_slice)
    km_center_id[ik] = days[np.argmin(D[0])]

In [ ]:
keys2remove = keys2remove

dict4dc = copy.deepcopy(dict_data2d_actual)
for key in keys2remove:
    del dict4dc[key]

dict4dc_key = list(dict4dc.keys())

km_centers_id_mat=np.empty(shape=(k,len(dict4dc_key)),dtype='int')
for i in range(len(dict4dc_key)):
    km_centers_id_mat[:,i]=km_center_id

err_dc_dict={}
for ikey, key in enumerate(dict4dc_key):
    data_replica = reconstruct_data(dict4dc[key],km_lab, km_center_id)
    err=err_of_dc_grouply(dict4dc[key],data_replica, km_lab, km_center_id, day_wise=False, normalized=True, use_normalizd_form=False)
  
    err_dc_dict[key] = err


err_regular = copy.deepcopy(err_dc_dict)
df_err=pd.DataFrame([err_regular])
df_err.to_csv('err_dc.csv')

df_centers_id = pd.DataFrame(km_centers_id_mat, columns=dict_of_data_keys_ACTUAL, index=np.arange(k))
df_centers_id.to_csv('center_id_km.csv')

df_label=pd.Series(km_lab, name='Labels')
df_label.to_csv('labels_km.csv')


labels = copy.deepcopy(km_lab)
center_id = copy.deepcopy(km_centers_id_mat)

for ikey, key in enumerate(dict_of_data_keys):
    actual_name = dict_pseudokey2actual[key]
    N, H = np.shape(dict_data_norm[key])

    if key=='load':
        dat2use = ts_load_MAXscaled
    else:
        dat2use = dict_data_minmax[key]

    data_reconstructed = reconstruct_data(dat2use, labels, center_id[:,ikey], center_val=None)
    data_reconstructed_rep = data_reconstructed[center_id[:,ikey],:]
  
    df = pd.DataFrame(data_reconstructed_rep, columns=np.arange(1,25), index=np.arange(1,k+1))
    
    # NOTE: create folder f'km_c2a_k{k}' 
    df.to_csv(f'km_c2a_k{k}'+f'/{actual_name}.csv')
  
    weights_of_rep = clust_size(labels)
    df_weights = pd.Series(weights_of_rep, name='weights_per_cluster')
    df_weights.to_csv('weights_per_cluster.csv')

# hc_ward_close2avg

In [86]:
k=20
w=0

In [ ]:
dict_of_data = copy.deepcopy(dict_data_norm)        #(dict_data_minmax)
keys2remove = keys2remove

for key in keys2remove:
    del dict_of_data[key]

dict_of_data_keys = list(dict_of_data.keys())
print('dict_of_data_keys: \n', dict_of_data_keys)


#grouped_highlevel_features = grouped_highlevel_features

dict_pseudokey2actual = {'load':'Load',
                         'solar':'Solar',
                          
                          'Center':'Blackspring', 
                         'CE': 'Halkirk',
                         'SW_1':'Blue_Trail',
                         'SW_2':'McBride',
                         
                         'wf_ext2': 'Castle_Rock_Wind_Farm',
                         'wf_ext3':'Kettles_Hill',
                         'wf_ext5':'Wintering_Hills',
                         'wf_ext6':'Enmax'}


dict_of_data_keys_ACTUAL = [dict_pseudokey2actual[x] for x in dict_of_data_keys]

print('dict_pseudokey2actual: \n', dict_pseudokey2actual)


ts_all = np.zeros((N,0))
for ikey,key in enumerate(dict_of_data_keys):
    ts_all = np.concatenate((ts_all,dict_of_data[key]),axis=1)

In [88]:
hc = AgglomerativeClustering(n_clusters=k).fit(ts_all)
hc_lab = hc.labels_

In [89]:
keys2remove = keys2remove
dict4dc = copy.deepcopy(dict_data2d_actual)
for key in keys2remove:
    del dict4dc[key]

dict4dc_key = list(dict4dc.keys())


ntot, htot = np.shape(ts_all)
hc_center = np.zeros((k, htot))
for ik in range(k):
    days = np.ix_(hc_lab==ik)[0]
    days = days.tolist()
  
    sp = ikey * 24
    ep = sp + 24

    hc_center[ik,sp:ep] = np.mean(ts_all[days,sp:ep],axis=0)


In [ ]:
hc_center_id = np.empty(k, dtype='int')
for ik in range(k):
    days = np.ix_(hc_lab==ik)[0]
    days = days.tolist()
    ts_slice = ts_all[days,:]
    
    D = pairwise_distances(np.reshape(hc_center[ik,:], newshape=(1,-1)), ts_slice)
    hc_center_id[ik] = days[np.argmin(D[0])]

In [90]:
keys2remove

['wf_ext1', 'wf_ext4']

In [ ]:
dict4dc = copy.deepcopy(dict_data2d_actual)
for key in keys2remove:
    del dict4dc[key]

dict4dc_key = list(dict4dc.keys())


hc_centers_id_mat=np.empty(shape=(k,len(dict4dc_key)),dtype='int')
for i in range(len(dict4dc_key)):
    hc_centers_id_mat[:,i]=hc_center_id


err_dc_dict={}
for ikey, key in enumerate(dict4dc_key):
    data_replica = reconstruct_data(dict4dc[key],hc_lab, hc_center_id)
    err=err_of_dc_grouply(dict4dc[key],data_replica, hc_lab, hc_center_id, day_wise=False, normalized=True, use_normalizd_form=False)
    err_dc_dict[key] = err

err_regular = copy.deepcopy(err_dc_dict)
df_err=pd.DataFrame([err_regular])
df_err.to_csv('err_dc.csv')

df_centers_id = pd.DataFrame(hc_centers_id_mat, columns=dict_of_data_keys_ACTUAL, index=np.arange(k))
df_centers_id.to_csv('center_id_hc.csv')

df_label=pd.Series(hc_lab, name='Labels')
df_label.to_csv('labels_hc.csv')


labels = copy.deepcopy(hc_lab)
center_id = copy.deepcopy(hc_centers_id_mat)

for ikey, key in enumerate(dict_of_data_keys):
    actual_name = dict_pseudokey2actual[key]
    N, H = np.shape(dict_data_norm[key])

    if key=='load':
        dat2use = ts_load_MAXscaled
    else:
        dat2use = dict_data_minmax[key]



    data_reconstructed = reconstruct_data(dat2use, labels, center_id[:,ikey], center_val=None)
    data_reconstructed_rep = data_reconstructed[center_id[:,ikey],:]
  
    df = pd.DataFrame(data_reconstructed_rep, columns=np.arange(1,25), index=np.arange(1,k+1))
    
    # NOTE: create folder f'hc_c2a_k{k}'
    df.to_csv(f'hc_c2a_k{k}'+f'/{actual_name}.csv')
  
    weights_of_rep = clust_size(labels)
    df_weights = pd.Series(weights_of_rep, name='weights_per_cluster')
    df_weights.to_csv('weights_per_cluster.csv')

# KmHC

In [103]:
k=20
w=24

In [98]:
def KMHC(ts_data,kmean_labels,k_main,distmat_of_data, w=24):
    # note: kmean_labels is a set of labels obtained from clustering ts_data into
    # k1 clusters 
    # k_main is the total number of clusters. Hence, k_main//k1 is the number of sub-clusters in the 
    # second step of KmHC, where hierarchical clustering with min-max linkage is used.
    center_id_main = np.empty(k_main, dtype='int')
    labels_main = np.empty(len(kmean_labels), dtype='int')
    
    k_kmean = len(set(kmean_labels))
    k_per_group = k_main // k_kmean
  
    for ik in range(k_kmean):
        days = np.ix_(kmean_labels==ik)[0]
        distmat_of_data_sliced = distmat_of_data[np.ix_(days,days)]
        data_sliced = ts_data[days]

        hc_lab_mat = HC_MinMax_klst(data_sliced, np.array([k_per_group]), 
                                    w=w, 
                                    flag_distmat=True,  
                                    given_distmat=distmat_of_data_sliced)

        hc_lab = hc_lab_mat[0,:]

        center_id_tmp = minmax_center_index_finder(hc_lab,distmat_of_data_sliced)

        for jk in range(k_per_group):
            actual_label = int(ik*k_per_group + jk)
            center_id_main[actual_label] = int(days[int(center_id_tmp[jk])])
            obj_idx = np.ix_(hc_lab==jk)[0]
            for idx in obj_idx:
                labels_main[int(days[idx])] = actual_label

    return (labels_main, center_id_main)

In [99]:
dict_of_data = copy.deepcopy(dict_data_norm)  
keys2remove = keys2remove

for key in keys2remove:
    del dict_of_data[key]

dict_of_data_keys = list(dict_of_data.keys())



dict_pseudokey2actual = {'load':'Load',
                         'solar':'Solar',
                          
                          'Center':'Blackspring', 
                         'CE': 'Halkirk',
                         'SW_1':'Blue_Trail',
                         'SW_2':'McBride',
                         
                         'wf_ext2': 'Castle_Rock_Wind_Farm',
                         'wf_ext3':'Kettles_Hill',
                         'wf_ext5':'Wintering_Hills',
                         'wf_ext6':'Enmax'}


dict_of_data_keys_ACTUAL = [dict_pseudokey2actual[x] for x in dict_of_data_keys]

ts_all = np.zeros((N,0))
for ikey,key in enumerate(dict_of_data_keys):
    ts_all = np.concatenate((ts_all,dict_of_data[key]),axis=1)

In [ ]:
dict_of_data_keys = list(dict_of_data.keys())
print('dict_of_data_keys: \n', dict_of_data_keys)

nkey = len(dict_of_data_keys)

dict_of_data_keys_idx = {}
for idx_mykey, my_key in enumerate(dict_of_data_keys):
    dict_of_data_keys_idx[my_key] = idx_mykey

N, H = np.shape(dict_of_data[dict_of_data_keys[0]])

In [101]:
ts_all_original = np.zeros((N,0))
ts_all_original_modified = np.zeros((N,0))

huge_val_seperator = np.ones((N,1))*(1e6)

for ikey,key in enumerate(dict_of_data_keys):
    ts_all_original = np.concatenate((ts_all_original,dict_of_data[key]),axis=1)
    ts_all_original_modified = np.concatenate((ts_all_original_modified, huge_val_seperator, dict_of_data[key]),axis=1)

In [102]:
# note that adding huge_val_seperator is necessary which was missed in KmHC. Otherwise, one can map a sequence of a data set
# onto a sequence of another data set! However, we cannot do that as they have different nature.
dtwmat_modified_w24 = cdist_dtw(ts_all_original_modified, global_constraint="sakoe_chiba", sakoe_chiba_radius=24)

In [ ]:

# km_lab: labels of data being clustered into k1 groups 
# NOTE: In the first stage of KmHC (as what presented in the original paper), one needs to
# cluster data into k1 groups. So, for 20 clusters, we cluster them first to k1 groups, and then, we apply hc(minmax)
# in each group

# k1=10 --> use kmeans to find km_lab

km = KMeans(n_clusters=10, random_state=0).fit(ts_all) # alternatively, it is better to run for 1000 rs, and choose the best one
# i.e. the one with lowest inertia

km_lab = km.labels_
KMHC_labels, KMHC_centroids_ID = KMHC(ts_all_original_modified,km_lab,k,dtwmat_modified_w24, w=24)

In [ ]:
centroid_id_mat = np.empty((k,len(dict_of_data_keys)), dtype='int')
for i in range(len(dict_of_data_keys)):
    centroid_id_mat[:,i]=KMHC_centroids_ID

In [ ]:
err_dc_dict={}
for ikey, key in enumerate(dict_of_data_keys):
    data_replica = reconstruct_data(dict_of_data[key],KMHC_labels, KMHC_centroids_ID)
    err=err_of_dc_grouply(dict_of_data[key],data_replica, KMHC_labels, KMHC_centroids_ID, day_wise=False, normalized=True, use_normalizd_form=False)
  
    err_dc_dict[key] = err


err = copy.deepcopy(err_dc_dict)
df_err=pd.DataFrame([err])
df_err.to_csv('err_dc.csv')

df_label=pd.Series(KMHC_labels, name='Labels')
df_label.to_csv('labels_KMHC.csv')

df_centers_id = pd.DataFrame(centroid_id_mat, columns=dict_of_data_keys_ACTUAL, index=np.arange(k))
df_centers_id.to_csv('center_id_KMHC.csv')

labels = copy.deepcopy(KMHC_labels)
center_id = copy.deepcopy(centroid_id_mat)

for ikey, key in enumerate(dict_of_data_keys):
    actual_name = dict_pseudokey2actual[key]
    N, H = np.shape(dict_data_norm[key])

    if key=='load':
        dat2use = ts_load_MAXscaled
    else:
        dat2use = dict_data_minmax[key]
  
  
  
    data_reconstructed = reconstruct_data(dat2use, labels, center_id[:,ikey], center_val=None)
    data_reconstructed_rep = data_reconstructed[center_id[:,ikey],:]
  
    df = pd.DataFrame(data_reconstructed_rep, columns=np.arange(1,25), index=np.arange(1,k+1))
    df.to_csv(f'KMHC_k{k}'+f'/{actual_name}.csv') # create the dir first
  
  
    weights_of_rep = clust_size(labels)
    df_weights = pd.Series(weights_of_rep, name='weights_per_cluster')
    df_weights.to_csv('weights_per_cluster.csv')